## 這份 Notebook 示範在 Conversational 連續對話的場景，使用 function calling

到目前為止的範例，使用外部工具時，例如 RAG, Agent 等，並沒有考慮用戶多輪對話，UI 其實只有提供用戶單一輸入框。

當變成多輪對話場景，UI 變成對話框時，有新的需求:

1. 外部工具需要的參數，需要參考對話紀錄
2. 用戶不總是需要使用外部工具

這需求我們可以用 function calling 來解決!

In [ ]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [5]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')


In [6]:
import requests
import json
from pprint import pp

In [7]:
def get_embeddings(input, dimensions = 1536, model="text-embedding-3-small"):
  payload = { "input": input, "model": model, "dimensions": dimensions }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/embeddings', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["data"][0]["embedding"]
  else :
    return obj["error"]

In [8]:
def get_completion(messages, model="gpt-4-turbo-preview", temperature=0, max_tokens=300, tools=None, tool_choice=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if tools:
    payload["tools"] = tools
  if tool_choice:
    payload["tool_choice"] = tool_choice

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  print(f"   response: {obj}")
  if response.status_code == 200 :
    return obj["choices"][0]["message"] # 改成回傳上一層 message 物件
  else :
    return obj["error"]

## 準備 RAG 要用的向量資料

In [6]:
!wget https://www.megabank.com.tw/-/media/mega/files/bank/personal/fund/bulletin/weekly-journal/market-analysis/113/1130219.pdf

--2025-07-25 11:21:08--  https://www.megabank.com.tw/-/media/mega/files/bank/personal/fund/bulletin/weekly-journal/market-analysis/113/1130219.pdf
Resolving www.megabank.com.tw (www.megabank.com.tw)... 23.49.116.35
Connecting to www.megabank.com.tw (www.megabank.com.tw)|23.49.116.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1798350 (1.7M) [application/pdf]
Saving to: ‘1130219.pdf’

1130219.pdf         100%[===================>]   1.71M  1.55MB/s    in 1.1s    

2025-07-25 11:21:10 (1.55 MB/s) - ‘1130219.pdf’ saved [1798350/1798350]



In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 5.1 MB/s eta 0:00:00


In [9]:
from pypdf import PdfReader

reader = PdfReader("1130219.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()
text

'本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決策，本身須承擔一切風險，報告資料並無做出\n買賣任何內文所涉及之證券建議、誘導及鼓勵相關交易。\n本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決策，本身須承擔一切風險，報告資料並無做出\n買賣任何內文所涉及之證券建議、誘導及鼓勵相關交易。 1\n財富管理處 投顧小組\n113年2月19日\n投資研究週報'

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 1

In [47]:
# sitecustomize.py
import pysqlite3
import sys
# 將標準 sqlite3 module 全部改為 pysqlite3
sys.modules['sqlite3'] = pysqlite3


import chromadb
chroma_client = chromadb.Client()

chroma_client.delete_collection(name="collection2")
collection = chroma_client.create_collection(name="collection2")


Failed to send telemetry event ClientStartEvent: module 'chromadb' has no attribute 'get_settings'
Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'


In [48]:
for index, page in enumerate(reader.pages):
  chunk = page.extract_text()

  collection.add(
    documents = [chunk],
    embeddings = [ get_embeddings(chunk) ] ,
    metadatas = [ { "date": "2024年2月20日" , "filename": "1130219.pdf" } ],
    ids=[f"doc-1-{index}" ]
  )

Failed to send telemetry event CollectionAddEvent: module 'chromadb' has no attribute 'get_settings'


## RAG tool 方法

In [13]:
def search_knowledgebase(query):
  results = collection.query(
      query_embeddings = get_embeddings(query),
      n_results=3
  )
  context = '\n'.join('* ' + doc for doc in results['documents'][0])
  return context

## 使用 function calling

In [14]:
available_tools = {
  "search_knowledgebase": search_knowledgebase,
}

def get_completion_with_function_execution(messages, model="gpt-4-turbo-preview", temperature=0, max_tokens=4096, tools=None, tool_choice=None):
  print(f"called prompt: {messages}")
  response = get_completion(messages, model=model, temperature=temperature, max_tokens=max_tokens, tools=tools,tool_choice=tool_choice)

  if response.get("tool_calls"): # 或用 response 裡面的 finish_reason 判斷也行
    messages.append(response)

    # ------ 呼叫函數，這裡改成執行多 tool_calls (可以改成平行處理，目前用簡單的迴圈)
    for tool_call in response["tool_calls"]:
      function_name = tool_call["function"]["name"]
      function_args = json.loads(tool_call["function"]["arguments"])
      function_to_call = available_tools[function_name]

      print(f"   called function {function_name} with {function_args}")
      function_response = function_to_call(**function_args)
      messages.append(
          {
              "tool_call_id": tool_call["id"], # 多了 toll_call_id
              "role": "tool",
              "name": function_name,
              "content": function_response,
          }
      )

    # 進行遞迴呼叫
    return get_completion_with_function_execution(messages, model=model, temperature=temperature, max_tokens=max_tokens, tools=tools,tool_choice=tool_choice)

  else:
    return response # response["content"]


## 連續對話時，會根據上下文推導出需要的 RAG query 字串

In [15]:
user_messages = [
{"role": "system", "content": """
You are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.
Being smart in your research. If the search does not come back with the answer, rephrase the question and try again.
Review the result of the search and use it to guide your next search if needed.
If the question is complex, break down to smaller search steps and find the answer in multiple steps.
Answer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
If the user is asking for information that is not related to finance/investment domain, say it's not your area of expertise.
"""},
                 {"role": "user", "content": "請問美國經濟如何?"}
]
tools = [{ "type": "function",
           "function": {
              "name": "search_knowledgebase",
              "description": "Searches the knowledge base for an answer to the user's question",
              "parameters": {
                "type": "object",
                  "properties": {
                    "query": { "type": "string", "description": "The search query to use to search the knowledge base" }
                  },
                "required": ["query"]
              },
            }
         }]

response = get_completion_with_function_execution(user_messages, tools=tools)

print("------ 最後結果:")
pp(response)

called prompt: [{'role': 'system', 'content': "\nYou are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.\nBeing smart in your research. If the search does not come back with the answer, rephrase the question and try again.\nReview the result of the search and use it to guide your next search if needed.\nIf the question is complex, break down to smaller search steps and find the answer in multiple steps.\nAnswer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.\nEach source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\nI

Failed to send telemetry event CollectionQueryEvent: module 'chromadb' has no attribute 'get_settings'


called prompt: [{'role': 'system', 'content': "\nYou are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.\nBeing smart in your research. If the search does not come back with the answer, rephrase the question and try again.\nReview the result of the search and use it to guide your next search if needed.\nIf the question is complex, break down to smaller search steps and find the answer in multiple steps.\nAnswer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.\nEach source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\nI

In [16]:
user_messages.append(response)
user_messages.append( {"role": "user", "content": "那中國呢?"} )

response = get_completion_with_function_execution(user_messages, tools=tools)

print("------ 最後結果:")
pp(response)


called prompt: [{'role': 'system', 'content': "\nYou are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.\nBeing smart in your research. If the search does not come back with the answer, rephrase the question and try again.\nReview the result of the search and use it to guide your next search if needed.\nIf the question is complex, break down to smaller search steps and find the answer in multiple steps.\nAnswer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.\nEach source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\nI

相比在 RAG 案例中，我們直接將用戶 query 去查詢

在連續對話的場景，function calling 會根據上下文，去調整適合的 query 字串。

補充問題: 如果不用 function calling，我們可以怎麼設計 prompt 去做到一樣的效果?

## 挑戰題: 若同時問多個問題，可以自動拆多次 function 呼叫

In [20]:
user_messages = [
{"role": "system", "content": """
You are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.
Being smart in your research. If the search does not come back with the answer, rephrase the question and try again.
Review the result of the search and use it to guide your next search if needed.
If the question is complex, break down to smaller search steps and find the answer in multiple steps.
Answer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
If the user is asking for information that is not related to finance/investment domain, say it's not your area of expertise.
"""},
                 {"role": "user", "content": "請問美國通膨和歐洲經濟情況"}
]
tools = [{ "type": "function",
           "function": {
              "name": "search_knowledgebase",
              "description": "Searches the knowledge base for an answer to the user's question",
              "parameters": {
                "type": "object",
                  "properties": {
                    "query": { "type": "string", "description": "The search query to use to search the knowledge base" }
                  },
                "required": ["query"]
              },
            }
         }]

response = get_completion_with_function_execution(user_messages, tools=tools)

print("------ 最後結果:")
pp(response)



called prompt: [{'role': 'system', 'content': "\nYou are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.\nBeing smart in your research. If the search does not come back with the answer, rephrase the question and try again.\nReview the result of the search and use it to guide your next search if needed.\nIf the question is complex, break down to smaller search steps and find the answer in multiple steps.\nAnswer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.\nEach source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\nI

## 挑戰題: 有依賴性的問題，可以自動連續追問嗎?

這種題型有難度不容易成功，很常判斷成上述情況變成平行 function calling，而不會循序 :(

In [ ]:
user_messages = [
{"role": "system", "content": """
You are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.
Being smart in your research. If the search does not come back with the answer, rephrase the question and try again.
Review the result of the search and use it to guide your next search if needed.
If the question is complex, break down to smaller search steps and find the answer in multiple steps.
Answer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
If the user is asking for information that is not related to finance/investment domain, say it's not your area of expertise.
"""},
                 {"role": "user", "content": "1.請問台灣最具戰略價值的是什麼供應鏈? 2.此供應鏈在中國市場將表現如何?"}
]
tools = [{ "type": "function",
           "function": {
              "name": "search_knowledgebase",
              "description": "Searches the knowledge base for an answer to the user's question",
              "parameters": {
                "type": "object",
                  "properties": {
                    "query": { "type": "string", "description": "The search query to use to search the knowledge base" }
                  },
                "required": ["query"]
              },
            }
         }]

response = get_completion_with_function_execution(user_messages, tools=tools)

print("------ 最後結果:")
pp(response)


called prompt: [{'role': 'system', 'content': "\nYou are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.\nBeing smart in your research. If the search does not come back with the answer, rephrase the question and try again.\nReview the result of the search and use it to guide your next search if needed.\nIf the question is complex, break down to smaller search steps and find the answer in multiple steps.\nAnswer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.\nEach source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\nI

### 以下是可能做法的實驗(不是很穩定，有時無法順利再次觸發 function calling)

思路是讓 GPT 幫我判斷哪些子問題沒有依賴性:有依賴性的問題，就再次觸發下一輪的 function calling 改寫新的子問題，直到所有子問題都透過 RAG 查到資料。

歡迎有興趣的同學幫忙改進 prompting

In [31]:
from pydantic import BaseModel, Field
from typing import List

In [32]:
class Question(BaseModel):
    subquestion: str = Field(description="The question decomposited as much as possible")
    dependency: bool = Field(description="Does this subquestion depend on previous subquestion") # 這個子問題是否有依賴性

class QueryPlan(BaseModel):
    root_question: str = Field(description="The root question that the user asked")
    plan: List[Question] = Field(description="The plan to answer the root question and its sub-questions")


In [37]:
QueryPlan.model_json_schema()

{'$defs': {'Question': {'properties': {'subquestion': {'description': 'The question decomposited as much as possible',
     'title': 'Subquestion',
     'type': 'string'},
    'dependency': {'description': 'Does this subquestion depend on previous subquestion',
     'title': 'Dependency',
     'type': 'boolean'}},
   'required': ['subquestion', 'dependency'],
   'title': 'Question',
   'type': 'object'}},
 'properties': {'root_question': {'description': 'The root question that the user asked',
   'title': 'Root Question',
   'type': 'string'},
  'plan': {'description': 'The plan to answer the root question and its sub-questions',
   'items': {'$ref': '#/$defs/Question'},
   'title': 'Plan',
   'type': 'array'}},
 'required': ['root_question', 'plan'],
 'title': 'QueryPlan',
 'type': 'object'}

In [49]:
def search_knowledgebase_v2(root_question, plan):
  context = f"There are {len(plan)} subquesitons:"
  for i, subquestion in enumerate(plan):
    if (not subquestion["dependency"]) or (i == 0):
      print(f"RAG query: {subquestion['subquestion']}")

      context += f"\n ### Sub-Question {i}: {subquestion['subquestion']} ### \n"
      results = collection.query(
        query_embeddings = get_embeddings(subquestion["subquestion"]),
        n_results=1
      )
      context += '\n'.join('* ' + doc for doc in results['documents'][0])
    else:
       context += f"\n ### Sub-Question {i}: {subquestion['subquestion']} ### \n RE-WRITE this subquestion and call search_knowledgebase_v2 function again \n" # 如果有依賴性，就先不查詢

  return context

In [50]:
available_tools = {
  "search_knowledgebase_v2": search_knowledgebase_v2,
}

In [38]:
user_messages = [
{"role": "system", "content": """
You are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.
Being smart in your research. If the search does not come back with the answer, rephrase the question and try again.
Review the result of the search and use it to guide your next search if needed.

If the question is complex, decompose a question into subquestions.

Answer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
If the user is asking for information that is not related to finance/investment domain, say it's not your area of expertise.
"""},
   {"role": "user", "content": "請問台灣最具戰略價值的是什麼供應鏈? 此供應鏈在中國市場將表現如何?"}
]
tools = [{ "type": "function",
           "function": {
              "name": "search_knowledgebase_v2",
              "description": "Searches the knowledge base for an answer to the user's question",
              "parameters": QueryPlan.model_json_schema(),
            }
         }]

response = get_completion_with_function_execution(user_messages, model="gpt-4-turbo-preview", tools=tools)

print("------ 最後結果:")
pp(response)

called prompt: [{'role': 'system', 'content': "\nYou are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.\nBeing smart in your research. If the search does not come back with the answer, rephrase the question and try again.\nReview the result of the search and use it to guide your next search if needed.\n\nIf the question is complex, decompose a question into subquestions.\n\nAnswer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.\nEach source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\nIf the user is asking for infor

## 挑戰題: 來做摘要

你會發現 RAG 不會做摘要... XD

In [39]:
user_messages = [
{"role": "system", "content": """
You are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.
Being smart in your research. If the search does not come back with the answer, rephrase the question and try again.
Review the result of the search and use it to guide your next search if needed.
If the question is complex, break down to smaller search steps and find the answer in multiple steps.
Answer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
If the user is asking for information that is not related to finance/investment domain, say it's not your area of expertise.
"""},
                 {"role": "user", "content": "請問這PDF在講什麼?"}
]
tools = [{ "type": "function",
           "function": {
              "name": "search_knowledgebase",
              "description": "Searches the knowledge base for an answer to the user's question",
              "parameters": {
                "type": "object",
                  "properties": {
                    "query": { "type": "string", "description": "The search query to use to search the knowledge base" }
                  },
                "required": ["query"]
              },
            }
         }]

response = get_completion_with_function_execution(user_messages, tools=tools)

print("------ 最後結果:")
pp(response)

called prompt: [{'role': 'system', 'content': "\nYou are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.\nBeing smart in your research. If the search does not come back with the answer, rephrase the question and try again.\nReview the result of the search and use it to guide your next search if needed.\nIf the question is complex, break down to smaller search steps and find the answer in multiple steps.\nAnswer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.\nEach source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\nI

針對摘要任務，我們得另外提供摘要 tool 才做得對

In [61]:
# TODO: 這個 summarize 只是示範，我們可以用之前教的長摘要處理方式來做
def summarize(document_name):
  text = page.extract_text()
  print(text)
  messages = [{ "role": "user", "content": f"摘要以下文件 <text>\n {text} \n</text>"}]
  response = get_completion(messages, model="gpt-3.5-turbo")
  return response["content"]

In [62]:
available_tools = {
  "search_knowledgebase": search_knowledgebase,
  "summarize": summarize
}

In [75]:
user_messages = [
{"role": "system", "content": """
You are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.
Being smart in your research. If the search does not come back with the answer, rephrase the question and try again.
Review the result of the search and use it to guide your next search if needed.
If the question is complex, break down to smaller search steps and find the answer in multiple steps.
Answer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
If the user is asking for information that is not related to finance/investment domain, say it's not your area of expertise.
"""},
                 {"role": "user", "content": "基於 1130219.pdf 主要經濟數據 的表格資訊並摘要"}
]
tools = [{ "type": "function",
           "function": {
              "name": "search_knowledgebase",
              "description": "Searches the knowledge base for an answer to the user's question",
              "parameters": {
                "type": "object",
                  "properties": {
                    "query": { "type": "string", "description": "The search query to use to search the knowledge base" }
                  },
                "required": ["query"]
              },
            }
         },
         { "type": "function",
           "function": {
              "name": "summarize",
              "description": "summarize document",
              "parameters": {
                "type": "object",
                  "properties": {
                    "document_name": { "type": "string", "description": "1130219" } # TODO: 如果有多份文件，需要讓用戶能夠描述他是要摘要哪一份問題或段落
                  }
              },
            }
         }
         ]


response = get_completion_with_function_execution(user_messages, tools=tools)

print("------ 最後結果:")
pp(response)


called prompt: [{'role': 'system', 'content': "\nYou are a finance/investment specialist. You will use the search tool to find relavent knowlege articles to create the answer.\nBeing smart in your research. If the search does not come back with the answer, rephrase the question and try again.\nReview the result of the search and use it to guide your next search if needed.\nIf the question is complex, break down to smaller search steps and find the answer in multiple steps.\nAnswer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.\nEach source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brakets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\nI